In [13]:
import json
import tqdm
import hashlib
from typing import (
    Union,
    Sequence
)
import math
import random
import sympy as sp
import glob
import pandas as pd

In [14]:

# function hub
def read_json_file(file_path): 
    ''' opens a json file, return its content save in a dictionary
    @input: file path to the json file
    @output: a dictionary of json file content
    '''
    file = open(file_path, 'r')
    values = json.load(file)
    file.close()
    return values

# main hash function
def hash_elems(*a):
    
    h = hashlib.sha256()
    h.update("|".encode("utf-8"))
    
    for x in a:

        if not x:
            # This case captures empty lists and None, nicely guaranteeing that we don't
            # need to do a recursive call if the list is empty. So we need a string to
            # feed in for both of these cases. "None" would be a Python-specific thing,
            # so we'll go with the more JSON-ish "null".
            hash_me = "null"

        elif isinstance(x, str):
            # strings are iterable, so it's important to handle them before the following check
            hash_me = x
        elif isinstance(x, Sequence):
             # The simplest way to deal with lists, tuples, and such are to crunch them recursively.
            hash_me = str(hash_elems(*x))           
        else:
            hash_me = str(x)
        h.update((hash_me + "|").encode("utf-8"))

    # Note: the returned value will range from [1,Q), because zeros are bad
    # for some of the nonces. (g^0 == 1, which would be an unhelpful thing
    # to multiply something with, if you were trying to encrypt it.)

    # Also, we don't need the checked version of int_to_q, because the
    # modulo operation here guarantees that we're in bounds.
    # return int_to_q_unchecked(
    #     1 + (int.from_bytes(h.digest(), byteorder="big") % Q_MINUS_ONE)
    # )

    return (int.from_bytes(h.digest(), byteorder="big") % (q - 1))



def is_valid_of_Zq(n):
    is_valid = True
    if isinstance(n, str):
        n = int(n) 
    if n < 0 or n >= q:
        is_valid = False
 
    return is_valid   


def is_valid_of_Zrp(n):
    is_valid = True
    if isinstance(n, str):
        n = int(n)    
    if n < 0 or n >= p:
        is_valid = False
    
    n_power = pow(n, q, p)    
    if not equals(n_power, 1):
        is_valid = False
    return is_valid


def equals(a, b):
    '''compares two values
    @input: two integers a, b 
    @output: True if a, b have same values, False otherwise
    '''
    return (a == b)

def mod_p(n):
    if isinstance(n, str):
        n = int(n) 
    return n % p

def get_production(*targets):
    
    product = 1
    for target in targets:
        if isinstance(target, str):
            target = int(target) 
        product = mod_p(product * target)
    return product
    

# the following code is adapted from GeeksforGeeks
def power_mod(x, y, p): 
    ''' do modular exponentiation. 
    @input: x - base, 
            y - exponent,
            m - modulus
    @output: integer result of x^y % m
    '''
    # Initialize result 
    res = 1
      
    # Update x if it is more than or equal to p 
    if(x >= p):
        x = x % p

    # positive exponential
    while (y > 0): 
          
        # If y is odd, multiply x with result 
        if (y & 1): 
            res = (res * x) % p
  
        # y must be even now 
        y = y >> 1; # y = y/2 
        x = (x * x) % p
    return res
    
def __miller_test(d, num):
    ''' find a odd number of d such that num - 1 = d * 2^r
    @input: d - a odd number that num - 1 = d * 2^r for r >= 1
            num - the number needs to be check against
    @output: True if num is prime, False if it's a composite
    '''
    # Pick a random number in [2..n-2] 
    # Corner cases make sure that n > 4 
    a = 2 + random.randint(1, num - 4)

    # Compute a^d % n 
    x = power_mod(a, d, num)
  
    if (x == 1 or x == num - 1): 
        return True
  
    # Keep squaring x while one of the following doesn't happen 
    # (i) d does not reach n-1 
    # (ii) (x^2) % n is not 1 
    # (iii) (x^2) % n is not n-1 
    while (d != num - 1): 
        x = (x * x) % num
        d *= 2
  
        if (x == 1): 
            return False; 
        if (x == num - 1): 
            return True
  
    # Return composite 
    return False

def is_prime(num, k): 
    ''' implements Miller-Rabin algorithm to test the primality of a number
    @input: num - a positive integer
            k - the number of iterations, impacting accuracy
    @output: True if it's a prime, False otherwise 
    '''
    # Corner cases 
    if (num <= 1 or num == 4): 
        return False
    if (num <= 3): 
        return True
  
    # Find r such that n = 2^d * r + 1 for some r >= 1 
    d = num - 1
    while (d % 2 == 0): 
        d //= 2
  
    # Iterate given number of 'k' times 
    for i in range(k): 
        if (__miller_test(d, num) == False): 
            return False
  
    return True

def is_divisor(a, b): 
    '''check if a is a divisor of b
    @input: a, b - positive integers 
    @output: True if a is a divisor of b, False otherwise'''
    return (a % b == 0)

def create_hash_parameter_list(i): 
    '''create a list of parameters needed for hash computation
    @input: i - the i-th trustee/guardian
            base-hash - given base hash code
    @output: a list of all the parameters needed, including base hash Q, Ki,j, hi,j
    '''
    # invalid index input
    if (i < 0 or i > 4): 
        raise ValueError('i should be within range of 0 - 4.')

    else:
        # declare variables
        coefficient_file_path = ''
        param_list = []

        #append base hash to list
        #param_list.append(base_hash)

        # get file name dynamically
        coefficients_file_path = (coefficients_path + str(i) + '.json')
        
        
        # read file 
        coefficients = read_json_file(coefficients_file_path)

        # get all the commitment values Ki,j and append to list 
        commitments = coefficients['coefficient_commitments']
        for commitment in commitments: 
            param_list.append(commitment)
        
        # get all the hi,j values and append to list
        proofs = coefficients['coefficient_proofs']
        for item in proofs:
            h_ij = item['commitment']  # h -> commitment
            param_list.append(h_ij)

        return param_list


### Green box 1.1 Baseline parameters - overview 

In [3]:
# TODO: needs to update paths to new results when the constants.json is given with non-null 


In [28]:
# file path configuration
dataset_path = 'results_100_ballots/'


constants_file_path = dataset_path+'constants.json'
context_file_path = dataset_path+'context.json'
ballot_file_path = dataset_path+'encrypted_ballots/ballot_ballot-57fd8bec-dcc8-11ea-aa71-f45c89ba671b.json'
ballot_folder_path = dataset_path+'encrypted_ballots/'
tally_file_path = dataset_path+'tally.json'
coefficients_path = dataset_path+'coefficients/coefficient_validation_set_hamilton-county-canvass-board-member-'
description_path = dataset_path+'description.json'
spoiled_ballots_folder_path = dataset_path+'spoiled_ballots'
device_path = dataset_path+'devices/device_268678284863259.json'


constants = read_json_file(constants_file_path)
context = read_json_file(context_file_path)
ballot = read_json_file(ballot_file_path)
tally = read_json_file(tally_file_path)
device = read_json_file(device_path)

# basic parameters
g = constants['generator']
p = constants['large_prime']
q = constants['small_prime']
r = constants['cofactor']

K = int(context['elgamal_public_key'])
num_of_guardian = context['number_of_guardians']
threshold = context['quorum']
base_hash = context['crypto_base_hash']
extended_hash = context['crypto_extended_base_hash']

In [18]:
p_expected = int(('''104438888141315250669175271071662438257996424904738378038423348328
3953907971553643537729993126875883902173634017777416360502926082946377942955704498
5420976148418252467735806893983863204397479111608977315510749039672438834271329188
1374801626975452234350528589881677721176191239277291448552115552164104927344620757
8961939840619466145806859275053476560973295158703823395710210329314709715239251736
5523840808458360487786673189314183384224438910259118847234330847012077719019445932
8662497991739135056466263272370300796422984915475619689061525228653308964318490270
6926081744149289517418249153634178342075381874131646013444796894582106870531535803
6662545796026324531037414525697939055519015418561732513850474148403927535855819099
5015804625681054267836812127850996052095762473794291460031064660979266501285839738
1435755902851312071248102599442308951327039250818892493767423329663783709190716162
0235296692173009397831714158082331468230007669177892861540060422814237337064629052
4377485454312723950024587358201266366643058386277816736954760301634424272959224454
4608279405999759391099775667746401633668308698186721172238255007962658564443858927
6348504157753488390520266757856948263869301753031434500465754608438799417919463132
99322976993405829119''').replace('\n', ''))

q_expected = pow(2, 256) - 189

### Green box 1.2 Baseline parameters - check against each election

In [19]:
error = False 

# check p and q
if not equals(p, p_expected):
    error = True 
    print("The actual p value doesn't equal to the expected. ")
if not equals(q, q_expected):
    error = True
    print("The actual q value doesn't equal to the expected. ")
    
# use Miller-Rabin algorithm to check the primality of p and q
# set iteration to run 50 times by default
DEFAULT_K = 50
if not is_prime(p, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(p, DEFAULT_K), var = "p"))
if not is_prime(q, DEFAULT_K): 
    error = True
    print('''It\'s verfied as {res} that {var} is a prime number.'''
      .format(res = is_prime(q, DEFAULT_K), var = "q"))

# check equation p - 1 = qr
if not equals(p - 1, q * r): 
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = equals(p - 1, q * r), equation = "p - 1 = q * r"))

# check q is not a divisor of r, and 1 < g < p
if is_divisor(q, r):
    error = True
    print('''It\'s verfied as {res} that {var1} is not a divisor of {var2}.'''
      .format(res = not(is_divisor(q, r)), var1 = "q", var2 = "r"))

# check 1 < g < p
is_within_range = ((g > 1) and (g < p))
if not is_within_range: 
    error = True
    print('''It\'s verfied as {res} that {cond}.'''
      .format(res = is_within_range, cond = "1 < g < p"))

# check g^q mod p = 1 
result_actual = power_mod(g, q, p)
result_expected = 1 
if not equals(result_actual, result_expected):
    error = True
    print('''It\'s verfied as {res} that {equation}.'''
      .format(res = str(equals(result_actual, result_expected)), 
              equation = 'g^q mod p = 1'))
if not error:
    print('[green box 1] success')


[green box 1] success


### Green box 2.1 Key Generation, confirm hash computation
calculate $c_i = H(Q,K_{i,0},K_{i,1},K_{i,2},...,K_{i, k-1},h_{i,0},h_{i,1},h_{i,2},...,h_{i,k-1})$ mod q

related file: context.json

coefficients files


modify to:  $c_{i,j} = H(Q,K_{i,j},h_{i,j})$ mod q

### Green box 2.2 - check: $g^{u_{ij}} mod p = h_{i,j}K^{c_i}_{i,j} mod p$

In [7]:
# unfinished, results are false

error = False
# uses double for-loop to check, 0 <= i < 5, 0 <= j < 3
for i in range(0, num_of_guardian): 
    coefficients = read_json_file(coefficients_path + str(i) + '.json')
    
    for j in range(0, threshold): 
        # get given values
        coefficient_proofs = coefficients['coefficient_proofs'][j]
        u_ij = int(coefficient_proofs['response']) # u, modify to new data field
        h_ij = int(coefficient_proofs['commitment']) # h, modify to new data field
        k_ij = int(coefficient_proofs['public_key'])
        c_ij = int(coefficient_proofs['challenge']) # c
        
        # computed values
        c_ij_computed = hash_elems(base_hash, k_ij, h_ij) % q
        
        # verify c values are computed correctly
        if not equals(c_ij, c_ij_computed):
            error = True
            #print(("The actual c_ij value does not equal to the expected. when i = {i}, j = {j}")
            #      .format(i = i, j = j))
            print("actual   " + str(c_ij))
            print("computed " + str(c_ij_computed))
        box_2_left = pow(g, u_ij, p)
        box_2_right = mod_p(mod_p(h_ij) * pow(k_ij, ci_list[i], p))
        
        # verify both sides of the equation equal to each other
        if not equals(box_2_left, box_2_right):
            error = True
            print("𝑔𝑢𝑖𝑗𝑚𝑜𝑑𝑝 != ℎ𝑖,𝑗𝐾𝑐𝑖𝑖,𝑗𝑚𝑜𝑑𝑝")

        
if not error:
    print("success")

actual   985941757942741440667939151541959156401297201627502171725567595554716972256
computed 45834717236686434456053917816183315284627303056344435664370579326434213674932


NameError: name 'ci_list' is not defined

### green box 3 & 4

In [ ]:
#TODO: flatten all the aggregated "if" statements, keep only one parameter in one statement

In [35]:

# entire ballot files
big_alpha_list = []
big_beta_list = []


for file in glob.glob(ballot_folder_path + "*.json"):
    
    ballot = read_json_file(file)
    error = False
    
    # green box 3
    contests = ballot['contests']
    for contest in contests:

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:


            object_id = ballot_selection['object_id']

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])
            big_alpha_list.append(alpha)
            big_beta_list.append(beta)

            a0 = int(ballot_selection['proof']['proof_zero_pad'])
            a1 = int(ballot_selection['proof']['proof_one_pad'])
            b0 = int(ballot_selection['proof']['proof_zero_data'])
            b1 = int(ballot_selection['proof']['proof_one_data'])
            c0 = int(ballot_selection['proof']['proof_zero_challenge'])
            c1 = int(ballot_selection['proof']['proof_one_challenge'])
            v0 = int(ballot_selection['proof']['proof_zero_response'])
            v1 = int(ballot_selection['proof']['proof_one_response'])

            # unused assignments, see if want to delete
            alpha_pow = pow(alpha, q, p)
            beta_pow = pow(beta, q, p)
            a0_pow = pow(a0, q, p)
            b0_pow = pow(b0, q, p)
            a1_pow = pow(a1, q, p)
            b1_pow = pow(b1, q, p)

            #print("a0:"+str(a0))
            #print("alpha_1:"+str(alpha_1))
            # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.


            if not is_valid_of_Zrp(alpha): 
                error = True
                print ("alpha is not in set Zrp.")
            
            if not is_valid_of_Zrp(beta): 
                error = True
                print ("beta is not in set Zrp.")
            
            if not is_valid_of_Zrp(a0): 
                error = True
                print ("a0 is not in set Zrp.")
                
            if not is_valid_of_Zrp(a1): 
                error = True
                print ("a1 is not in set Zrp.")
            
            if not is_valid_of_Zrp(b0): 
                error = True
                print ("b0 is not in set Zrp.")
                
            if not is_valid_of_Zrp(b1): 
                error = True
                print ("b1 is not in set Zrp.")

            # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
            # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
            # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
            c = hash_elems(extended_hash, alpha, beta, a0, b0, a1, b1)
            #print(c)

            #print((c0 + c1) % q)

            if not equals(c, (c0 + c1) % q):
                error = True
                print("c != (c0 + c1) % q.")

            # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
            if not is_valid_of_Zq(c0):
                error = True 
                print("c0 is not in the set of Zq. ")
               
            if not is_valid_of_Zq(c1):
                error = True 
                print("c1 is not in the set of Zq. ")
                
            if not is_valid_of_Zq(v0):
                error = True
                print("v0 is not in the set of Zq. ")
                
            if not is_valid_of_Zq(v1):
                error = True
                print("v1 is not in the set of Zq. ")

            # 1st equation: gv0=a0alphac0 mod p
            g_v0_ls = pow(g, v0, p)
            g_v0_rs = mod_p(a0 * pow(alpha, c0, p))

            # 2nd equation: gv1=a1alphac1 mod p
            g_v1_ls = pow(g, v1, p)
            g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

            # 3rd equation: Kv0=b0betac0 mod p
            K_v0_ls = pow(K, v0, p)
            K_v0_rs = mod_p(b0 * pow(beta, c0, p))

            # 4th equation: gc1Kv1=b1betac1 mod p
            fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
            fourth_rs = mod_p(b1 * pow(beta, c1, p))

            # checking
            if not equals(g_v0_ls,g_v0_rs):
                print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
                error = True
            if not equals(g_v1_ls, g_v1_rs):
                print("g_v1 != a_1*alpha^c_1")
                error = True
            if not equals(K_v0_ls, K_v0_rs):
                print("K_v0 != b_0*alpha^c_0")
                error = True
            if not equals(fourth_ls, fourth_rs):
                print("g_c1 * K_v1 != (b1 * beta^c1) % p")
                error = True


    if not error:
        print("[green box 3 success] " + file)  
    
    # green box 4
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1
        big_ai = 1
        big_bi = 1

        if not is_valid_of_Zq(big_v):
            error = True
            print("V is not in the set of Zq. ")
        
        if not is_valid_of_Zrp(big_a):
            error = True
            print ("A is not in the set of Zrp. ")
            
        if not is_valid_of_Zrp(big_b):
            error = True
            print("B is not in the set of Zrp. ")

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)
            
            
            ai_zero = int(ballot_selection['proof']['proof_zero_pad'])
            ai_one = int(ballot_selection['proof']['proof_one_pad'])
            bi_zero = int(ballot_selection['proof']['proof_zero_data'])
            bi_one = int(ballot_selection['proof']['proof_one_data'])
            big_ai = mod_p(big_ai * ai_zero)
            big_ai = mod_p(big_ai * ai_one)
            big_bi = mod_p(big_bi * bi_zero)
            big_bi = mod_p(big_bi * bi_one)

        
        # bullet point 2 
        # contest total(A,B) 
        print(big_a)
        print(big_ai)
        print(big_alpha)
        
        #print(big_b)
        #print(big_beta)
        
        """
        temp_c = hash_elems(extended_hash, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if not equals(g_v_ls, a_ac_rs):
            error = True
            print("g_v_ls does not equal to a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if not equals(g_l_k_ls, b_bc_rs):
            error = True
            
            print("g_l_k_ls does not equal to b_bc_rs.")        
        """
    if not error:
        print("[green box 4 success] " + file)
      
    """
    # green box 5
    timestamp = ballot['timestamp']
    ballot_id = ballot['description_hash']
    prev_tracking_hash = ballot['previous_tracking_hash']
    tracking_hash = ballot['tracking_hash']
    crypto_hash = ballot['crypto_hash']
    
    device_id = hash_elems(str(device['uuid']), device['location'])    
    temp_tracking_hash = hash_elems(prev_tracking_hash, timestamp, crypto_hash)
    if not equals(tracking_hash, temp_tracking_hash):
        error = True
    if not error:
        print("[green box 5 success] " + file)    
    """
    print("-------------------------------------------------------------------------------")

    
        
        
    

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-5808f32e-dcc8-11ea-aa71-f45c89ba671b.json
39922696811794868839903777387700167716653997346038871064333731157731925316216915389801229057993351975497976590272031702134043875258014618706280834288926962349589575025915718169441553955710257455849700095717018482727468536289141836267570527079917834241776316340056518592515807866301479445559468100203674538931054347457331685406366356649251551100047751871741563155133221707560944238845597619666640487555207372711750144496369342359951948601576391765779628583302999520609025709337356465930345932639018227399657978194620833929232836529332776524225947449636292987624859638265456118680472321893767236410758121777389017082993533213565204746117524797708255516161928258465155138468784849681272152485909484322968569740083960117874488105443665948726789081478976790526666206106016152952857126976624921487367995117260920219277653066196341365552350761201587297145750653746746723625398794586445345836

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-57fdd1ba-dcc8-11ea-aa71-f45c89ba671b.json
44640219457016794892884902495721847343941935414521014317356888372761997186949324150579649443238914282905836000152909191190850889872619954891619743073269127489875981097597455992114648663949182643366477813976776100269748046976049983945079606486070053099073653577307391631082166380876087784667496513430222615110952148109624252916824991749836341780991448763217748375932396865132832957766229093894128938540853096700165719044823365095026745795172020046491016344924018499414814376929415215551896873972568691284299371445809224126208932076812162865943081450372447425058353058656212705493642757555720155879021834483822215484221174369808812586360151570433903171295601943358867928787479671570187621640228535514678992593673081732039846209592450438441389072952152267580936021844807448403117106547984116007397943146661058533490383785070018880444570283275918314338772234516778453615126794713244329560

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-58070afa-dcc8-11ea-aa71-f45c89ba671b.json
51736747416573019155268040623619844065689731754839507889497632992510050102400953460561816319134804062336790621537394799687953220832550779836990257721530846755794189992105006790057161856866488748436962877249120684730147934319166693278021896473368995539934120261960812926845396985152079151946188793335356128845665772618009237654193270484811767147979474185372627658223407865762803863219948441458451543881922798146974878713245629143277697244205997968566901486545634543046077231632500729213045687477919208601027268285111307300158022075429118638180284857555282091052484042191115854170642039380977977495658185027925742870021497626969865764554124438091141470192554258561744394642011855889984624555372487493914583654301197812001989829314633541131113517341705513891964655155989552191444125906864959594755096912532431483650798698355665019998919847100436557187502569143556415695344355962462395364

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-57fe8678-dcc8-11ea-aa71-f45c89ba671b.json
50653486986563925099388200967015679147922474577179104306108374095731099680820037715045128676802266708340233659488437837295041364726364575511467662555253327280791156237382344103408320010792788037960744272783990596354749406451408032612098650618715104773500286881102330980515828151478402057251271192792134331441910332870359246304561067258780338618881694850797186573175946039088670758687305372727861958985048097973642405339048333194307914636242773740250807392248086837128551134445313780295177560326129815763424088340064160360898498713606862842991441700855708464528841720752502184357288435495098705867181379102649001410014500296074275902418814094304164096341244968470819527471176479356055733087319525374171304801322670714174306024179622984334431245280939752457478892764848276220468174213806094584185114849318516122517817069972425768888338023956531716887321071837118960772421419267791275567

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-58015132-dcc8-11ea-aa71-f45c89ba671b.json
71674288700390440512535061610590293333917291630140739627754215330594079235810828242935761817494328094816674930493097406544799311170109053632491112600319172794658110272532694211437400302513414582153864085294774401607170470363957039217501570149854024148502030576869442945381245593226141812595165436422533903567085506677470390304411508708398459644260023752237699824896154768200859910303032887786019637474235642184364961056197747349473417357799214321414752626242234220655187114139173488637813587451405432423769315573185573297848573820147855839243688735399495193167544750927050928220133067211832013751208498521954732327752049958392749633084616592645197915316270585402600270666135088659304302058187798440465114532932456462996247705072225236580389598729924262235230808770900773797578981434307541256100816254373805474192831012333145592799985835229549600568644700179252437552826466344466818338

...

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-5806b78a-dcc8-11ea-aa71-f45c89ba671b.json
33946169093662093901650964995578521132208134330585237582407871227194389267503578393091934825571601730318895073527647307697662383016877018148428030459265391885907352999510660065859829424242993052809674825742711629826751167651124482545518376215363443568001459161863889227542390075296358614580320431701961180640608332113104960753555242697022294036455599565279740315651429748623117851311064297389632514288353503356905070502587440907940384952551764270713710056502650840071093437415796110040842713562353928950659481995812068957429349505220219803709582411876940339390920614762300763025743780284289181013363672719042726539418263328379722591642403129333054556373031096927595077004966862445333988451542385513729469338584041625608346525656550240509009743068668372792118322498993921525913057920493103965286779454075793579928981136771227770856415518083994143615296883146499046345610458554694221267

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-580d0522-dcc8-11ea-aa71-f45c89ba671b.json
80479235499803369101028217565457386511983369636525338040871074693937714857244747288704832297596863675469227070050890694294753447515220814259115058574449440203650180048463715875643378067189225799467546345489578679942739373681321120259379239736169861987884429054447750418721097245360212240695994385951177105918912128504030654499110080358645161820847317749832612852935251181554468713057309872660552519380352928372512238120147582885346313219678448439010926475313846251604868338847710063733886426765052461088171185632500457539169690194211115097280421327026882716125269502520447940302140724431983593939194926286301993964442412253973183262800446033806033064955619600808184655439984365887180216649073682273818706923709675003770945501170191563709280584688890445464746538871950770223272133981422415277506672870905899079959768682220570230926395898337879050133008082118466791623015087105104102536

[green box 3 success] results_100_ballots/encrypted_ballots/ballot_ballot-5801db52-dcc8-11ea-aa71-f45c89ba671b.json
24879098402159461648863846135837841521516428328549778487617218817644084665938421369035371541051522902613830005524765026849143667029986712560357629940165367307812340029058727120183088299567718628482590616889604546026759654537373255986794018197469886268862605838272655447964763949376749331146223137519925798845216029855277191141908429489576369968337612280604274626921985240779502834909414075456899572903445343691578558192048541784292190244513913974225777204805864636641844506203536755257981485120882211380185285837109411168360058047793968068141988505450507450774298189721160262385698089169840272154096901290854311247638649674775711727194152894039485862927715575807814624192599190427776313469997421379840594562644790096829362785204065422929254988587214436261372524829495921866581484211974737146948270556803267076747379356188505849452074506635388689173062446425192988923053113733992254008

7633248918087898962196471147758777320774907212615225123981690160695217271328350146904870691819849883918300955074946056275995210948743316453626950321632661341847199089809667805178009752239456838078509810708133394102850968729987035204408455294711018646922770233660946911971633645576120747494701165966328461106492366054233815190169671612452036593372181522148123778849548865180160892908593859663244716352383285611289889715249400073409106592127656736417001520643645205906692901091746832217107864131366350769030302672665641646795079551595609232967872487399721021534629640679010694124452534901101256618398761689713256489294899361997816528747538027817846567818658647666988297943579811749151030414774066778062322201319008308100721226916435549946456801691479900275046488279376017227866819747754616142040744440679078937627006579868965994503079112484322972440308341356230008670589160693699594039239768902846203633761332689860643597591700925567448566756702866915751084688176461481562397445986224159521478305638459

KeyboardInterrupt: 

In [110]:

# one sample data
ballot = read_json_file(ballot_file_path)

error = False

contests = ballot['contests']
for contest in contests:
    
    ballot_selections = contest['ballot_selections']
    for ballot_selection in ballot_selections:


        objectId = ballot_selection['object_id']

        alpha = int(ballot_selection['ciphertext']['pad'])
        beta = int(ballot_selection['ciphertext']['data'])

        a0 = int(ballot_selection['proof']['proof_zero_pad'])
        a1 = int(ballot_selection['proof']['proof_one_pad'])
        b0 = int(ballot_selection['proof']['proof_zero_data'])
        b1 = int(ballot_selection['proof']['proof_one_data'])
        c0 = int(ballot_selection['proof']['proof_zero_challenge'])
        c1 = int(ballot_selection['proof']['proof_one_challenge'])
        v0 = int(ballot_selection['proof']['proof_zero_response'])
        v1 = int(ballot_selection['proof']['proof_one_response'])

        # unused assignments, see if want to delete
        alpha_pow = pow(alpha, q, p)
        beta_pow = pow(beta, q, p)
        a0_pow = pow(a0, q, p)
        b0_pow = pow(b0, q, p)
        a1_pow = pow(a1, q, p)
        b1_pow = pow(b1, q, p)

        #print("a0:"+str(a0))
        #print("alpha_1:"+str(alpha_1))
        # 3.1 The given values alpha, beta, a0, b0, a1, and b1 are all in the set Zpr.
        
        
        if not is_valid_of_Zrp(alpha): 
            error = True
            print ("alpha is not in set Zrp.")
            
        if not is_valid_of_Zrp(beta): 
            error = True
            print ("beta is not in set Zrp.")
            
        if not is_valid_of_Zrp(a0): 
            error = True
            print ("a0 is not in set Zrp.")
                
        if not is_valid_of_Zrp(a1): 
            error = True
            print ("a1 is not in set Zrp.")
            
        if not is_valid_of_Zrp(b0): 
            error = True
            print ("b0 is not in set Zrp.")
                
        if not is_valid_of_Zrp(b1): 
            error = True
            print ("b1 is not in set Zrp.")
            
        # 3.2 The challenge c is computed as c = H(Q, (alpha, beta), (a0, b0), (a1, b1))
        # tempString = str(q_bar) + str(alpha) + str(beta) + str(a0) + str(b0) + str(a1) + str(b1)
        # tempList = (q_bar, (alpha, beta), (a0, b0), (a1, b1))
        c = hash_elems(extended_hash, alpha, beta, a0, b0, a1, b1)
        #print(c)
        
        #print((c0 + c1) % q)
        
        if not equals(c, (c0 + c1) % q):
            print("c != (c0 + c1) % q.")
            error = True

        # 3.3 The given values c0, c1, v0, and v1 are each in the set Zq.
        if not is_valid_of_Zq(c0):
            error = True 
            print("c0 is not in the set of Zq. ")
               
        if not is_valid_of_Zq(c1):
            error = True 
            print("c1 is not in the set of Zq. ")
                
        if not is_valid_of_Zq(v0):
            error = True
            print("v0 is not in the set of Zq. ")
                
        if not is_valid_of_Zq(v1):
            error = True
            print("v1 is not in the set of Zq. ")        

        # 1st equation: gv0=a0alphac0 mod p
        g_v0_ls = pow(g, v0, p)
        g_v0_rs = mod_p(a0 * pow(alpha, c0, p))
    
        # 2nd equation: gv1=a1alphac1 mod p
        g_v1_ls = pow(g, v1, p)
        g_v1_rs = mod_p(a1 * pow(alpha, c1, p))

        # 3rd equation: Kv0=b0betac0 mod p
        K_v0_ls = pow(K, v0, p)
        K_v0_rs = mod_p(b0 * pow(beta, c0, p))
    
        # 4th equation: gc1Kv1=b1betac1 mod p
        fourth_ls = mod_p(pow(g, c1, p) * pow(K, v1, p))
        fourth_rs = mod_p(b1 * pow(beta, c1, p))

        # checking
        if not equals(g_v0_ls,g_v0_rs):
            print("g_vo != a_0*alpha^c_0 at cast_ballots {i}, contest {j}, selection {k}")
            error = True
        if not equals(g_v1_ls, g_v1_rs):
            print("g_v1 != a_1*alpha^c_1")
            error = True
        if not equals(K_v0_ls, K_v0_rs):
            print("K_v0 != b_0*alpha^c_0")
            error = True
        if not equals(fourth_ls, fourth_rs):
            print("g_c1 * K_v1 != (b1 * beta^c1) % p")
            error = True

                                               
if not error:
    print("success")   



success


### TODO: (box 4)
1. modify variable names -> make sure to use underline instead of camal cases
2. change function names accordingly
3. flatten if statements

In [12]:
# Green box 4

green_box_five_verification_list = []

temp_hash = hash_elems(extended_hash)
green_box_five_verification_list.append(temp_hash)


for file in glob.glob(ballot_folder_path + "*.json"):
    
    ballot = read_json_file(file)

    error = False

    contests = ballot['contests']
    for contest in contests:

        # total parameters
        big_a = int(contest['proof']['pad'])
        big_b = int(contest['proof']['data'])
        big_c = int(contest['proof']['challenge'])
        big_v = int(contest['proof']['response'])
        big_constant = int(contest['proof']['constant'])
        big_alpha = 1
        big_beta = 1
        


        if not is_valid_of_Zq(big_v):
            error = True
            print("V is not in the set of Zq. ")
        
        if not is_valid_of_Zrp(big_a):
            error = True
            print ("A is not in the set of Zrp. ")
            
        if not is_valid_of_Zrp(big_b):
            error = True
            print("B is not in the set of Zrp. ")

        ballot_selections = contest['ballot_selections']
        for ballot_selection in ballot_selections:

            alpha = int(ballot_selection['ciphertext']['pad'])
            beta = int(ballot_selection['ciphertext']['data'])  
            big_alpha = mod_p(big_alpha * alpha)
            big_beta = mod_p(big_beta * beta)          
            
        temp_c = hash_elems(extended_hash, big_alpha, big_beta, big_a, big_b)
        #print(temp_c)
        #print(big_c)


        # g power v mod p = a * A power c mod p
        g_v_ls = pow(g, big_v, p)
        a_ac_rs = mod_p((mod_p(big_a)) * pow(big_alpha, big_c, p))

        #print(g_v_ls)
        #print(a_ac_rs)
        if g_v_ls != a_ac_rs:
            error = True
            print("g_v_ls is not equal a_ac_rs.")


        if big_constant is None:
            constant_q = 0
        else:
            constant_q = big_constant

        # g power Lc mod p * K power v mod p = b * B power c mod p
        g_l_k_ls = mod_p(pow(g, mod_p(constant_q * big_c), p) * pow(K, big_v, p))
        b_bc_rs = mod_p(mod_p(big_b) * pow(big_beta, big_c, p))


        # mult_p(g_pow_p(mult_p(c, constant_q)), pow_p(k, v))
        # mult_p(b, pow_p(beta, c))
        #print(g_l_k_ls)
        #print(b_bc_rs)
        if g_l_k_ls != b_bc_rs:
            error = True
            print("g_l_k_ls is not equal b_bc_rs.")   
    
    temp_hash = hash_elems(green_box_five_verification_list[-1], device['uuid'], ballot['timestamp'], ballot['object_id'])
    print(temp_hash)
    print(ballot['crypto_hash'])
    print(ballot['tracking_hash'])
    print("----------------------------------------------------------------------")
    
    green_box_five_verification_list.append(temp_hash)

    if not error:
        print("success")   

66282747550593521138008278310118601600647030566586190627122017498891695970886
75573491678308373158265710038357347131424982572707387031276846583598508703954
100331963551170212562260317515169406221302018484683647350873406023228476002853
----------------------------------------------------------------------
success
3572193811405051617267211616797916547207464294683735746610949085294613360286
2605657608689518340104917398376402479652710419974208354454495191224843560911
10177604193855882303518051054396675826180501749803479545421861608412549973420
----------------------------------------------------------------------
success
42073926292243688609770133207961679239337380403077914665316667732386956234962
748921653608524887668295704647830027278389245070492777092974253557664051905
71251228239524275331892730087322319153300935546885368860778876190680731052932
----------------------------------------------------------------------
success
8385810602865906657362618827368264491639601315344203442076926258

12678132348222150036066553668339790046993864823966860145285960693538381456050
83406746017777039547849685991571894254217883599234143640890809821580085267591
23320311929661854783342829258204011127607289604738097954313790882558612528406
----------------------------------------------------------------------
success
75162354934440278779122582040205687633455295046303108679545375213788833519758
114656674042547264895076553742177759322930487892081651973090129927995159382839
21714055068929732316240435091676797988952930124798682721041558040004436374541
----------------------------------------------------------------------
success
35290120618834472558357990885500101891056138978394182961175114472244410854701
8009541610920308617820037961121426384937825054484401237590594923897307189706
65624151431587456313529768768643591520491874704034647711379669541175079133150
----------------------------------------------------------------------
success
5620508109238924895468182187811673060002326367735423996492207

39428422459906952344288546228555661618852163841903006258469576981047119988937
18660252305751010955606378484938061267675147449198364167915360333324312902083
18747922354391978181293085197655504414512448773672244220434358301632793930167
----------------------------------------------------------------------
success
6633968243077857874773403670267597229637710140824010011961772854688496368404
74340223613897854753384725918890959528421685093723228411323770166897187630974
73651833904093917196045655413895390228001169794528938882076301748829602831013
----------------------------------------------------------------------
success
103251133194410404054343988610986977436572468820132143113512570120137326205556
15802027604905764667644540103143406656599445628757640633250403204559255183149
89066735960962990542220292664808400545667486127525136110875767525334505723870
----------------------------------------------------------------------
success
3679470134512342923781415322895985639550541617923168334964271

23764650207615329846821422208819159365347311532161572339905263328516076469741
71404944149920048844177572553004010498143674557554366491254904544782448975937
83999255644532248093734057586214251089994955367507198278051596302246575001020
----------------------------------------------------------------------
success
32425552073712280688158550564874355050341217238583924571105416522529173563398
89044257924046378768844545250371159740100655952049961491115867770347638990867
78740805717096849276456734489950526634453200042962786600812226136025801184648
----------------------------------------------------------------------
success
106799058716771026547534535127827312151559159564236432456051233584138025257065
80829371344866576225410700195776457415428676748241243256034158948228538217311
40314357966191493302989229808468969835216638059793513978341511721421710927898
----------------------------------------------------------------------
success
108323328161928694789484219361445665190730576845087931254709

### green box 6 - decryption

In [91]:
# TODO: verify big_A, big_B by getting products from 100 ballot files

# create data frames with column names 

selection_name_lists = []

def switch_idx(contest_name): 
    switcher = {
        'president-vice-president-contest': 0,
        'ozark-governor': 1,
        'congress-district-5-contest': 2,
        'congress-district-7-contest': 3,
        'pismo-beach-school-board-contest': 4,
        'somerset-school-board-contest': 5,
        'arlington-chief-justice-retain-demergue': 6,
        'exeter-utility-district-referendum-contest': 7,
    }
    
    if (contest_name in switcher.keys()): 
        return switcher[contest_name]
    else: 
        return None


description = read_json_file(description_path)
contests = description['contests']

for i in range(len(contests)):
    selection_name_lists.append(['id'])
    
for i in range(len(contests)):
    contest = contests[i]
    contest_name = contest['object_id']
    selections = contest['ballot_selections']
    curr_list = selection_name_lists[switch_idx(contest_name)]
    for selection in selections: 
        selection_name = selection['object_id']
        if not selection_name in curr_list: 
            curr_list.append(selection_name + '-a')
            curr_list.append(selection_name + '-b')

In [92]:
print(selection_name_lists)

[['id', 'barchi-hallaren-selection-a', 'barchi-hallaren-selection-b', 'cramer-vuocolo-selection-a', 'cramer-vuocolo-selection-b', 'court-blumhardt-selection-a', 'court-blumhardt-selection-b', 'boone-lian-selection-a', 'boone-lian-selection-b', 'hildebrand-garritty-selection-a', 'hildebrand-garritty-selection-b', 'patterson-lariviere-selection-a', 'patterson-lariviere-selection-b', 'write-in-selection-president-a', 'write-in-selection-president-b'], ['id', 'franz-selection-a', 'franz-selection-b', 'harris-selection-a', 'harris-selection-b', 'bargmann-selection-a', 'bargmann-selection-b', 'abcock-selection-a', 'abcock-selection-b', 'steel-loy-selection-a', 'steel-loy-selection-b', 'sharp-selection-a', 'sharp-selection-b', 'walace-selection-a', 'walace-selection-b', 'williams-selection-a', 'williams-selection-b', 'alpern-selection-a', 'alpern-selection-b', 'windbeck-selection-a', 'windbeck-selection-b', 'sharp-althea-selection-a', 'sharp-althea-selection-b', 'greher-selection-a', 'greher-

In [93]:
dfs = []

for i in range(len(selection_name_lists)):
    df = pd.DataFrame(columns = selection_name_lists[i])
    dfs.append(df)

# read all ballot files
# unfinished
for file in glob.glob(ballot_folder_path + "*.json"):
    ballot = read_json_file(file)
    ballot_id = ballot['object_id']
    ballot_state = ballot['state']
    contests = ballot['contests']
    
    # ignore spoiled ballots
    if (ballot_state == 'CAST'): 
        for i in range(len(contests)):
            contest = contests[i]
            contest_name = contest['object_id']
            curr_df = dfs[switch_idx(contest_name)]
            selections = contest['ballot_selections']
            # store data from a ballot
            curr_list = []
            curr_list.append(ballot_id)
            for selection in selections:
                is_placeholder = selection['is_placeholder_selection']
                if not is_placeholder: 
                    selection_name = selection['object_id']
                    a = selection['ciphertext']['pad']
                    b = selection['ciphertext']['data']
                    curr_list.append(a)
                    curr_list.append(b)
            # assign to a row and add to a designated dataframe
            curr_df_leng = len(curr_df)
            curr_df.loc[curr_df_leng] = curr_list
                    

In [94]:
# check number of cast ballots of each contest
for df in dfs:
    print(df.shape)

(92, 15)
(92, 55)
(34, 13)
(58, 13)
(16, 21)
(17, 17)
(45, 5)
(11, 5)


In [95]:
def get_product_mod_p(dataframe, column_index):
    product = 1
    column = dataframe.iloc[:, column_index]
    
    for i in range(len(column)):

        product *= int(column.loc[i]) % p
        
    return product

In [106]:
# for each non-dummy option in each cas ballot
# verify A = alpha_j product, B = beta_j product

contest_names = list(tally.keys())
# loop through contests
for contest_name in contest_names:
    error = False

    # match tally with specific dataframe
    curr_df = dfs[switch_idx(contest_name)]
    
     # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    idx = 0
    
    # loop through selections
    for selection_name in selection_names:
        selection = tally[contest_name]['selections'][selection_name]
        
        # get A and B
        big_alpha = mod_p(int(selection['message']['pad']))
        big_beta= mod_p(int(selection['message']['data']))
        
        # get alpha_j and beta_j products
        alpha_j_product = mod_p(get_product_mod_p(curr_df, 2 * idx + 1))
        beta_j_product = mod_p(get_product_mod_p(curr_df, 2 * idx + 2))
        
        if not equals(alpha_j_product, big_alpha):
            error = True
            print(selection_name + " alpha error")
        if not equals(beta_j_product, big_beta):
            error = True
            print(selection_name + " beta error")
        if equals(alpha_j_product, big_alpha) and equals(beta_j_product, big_beta):
            print(selection_name + " sucess")
        idx += 1
        
if not error:
    print("all contsets success")

barchi-hallaren-selection sucess
cramer-vuocolo-selection sucess
court-blumhardt-selection sucess
boone-lian-selection sucess
hildebrand-garritty-selection sucess
patterson-lariviere-selection sucess
write-in-selection-president sucess
franz-selection sucess
harris-selection sucess
bargmann-selection sucess
abcock-selection sucess
steel-loy-selection sucess
sharp-selection sucess
walace-selection sucess
williams-selection sucess
alpern-selection sucess
windbeck-selection sucess
sharp-althea-selection sucess
greher-selection sucess
alexander-selection sucess
mitchell-selection sucess
lee-selection sucess
ash-selection sucess
kennedy-selection sucess
jackson-selection sucess
brown-selection sucess
teller-selection sucess
ward-selection sucess
murphy-selection sucess
newman-selection sucess
callanann-selection sucess
york-selection sucess
chandler-selection sucess
write-in-selection-governor sucess
soliz-selection sucess
keller-selection sucess
rangel-selection sucess
argent-selection suc

In [116]:
# green box 6 - equations

# get all ki_0

k_i0_list = []
    
for i in range(0, num_of_guardian): 
    coefficients = read_json_file(coefficients_path + str(i) + '.json')
    k_i0 = int(coefficients['coefficient_commitments'][0])
    k_i0_list.append(k_i0)
    
# get all the contest names stored in the first layer
contest_names = list(tally.keys())

# loop over each contest 
for contest_name in contest_names: 
    error = False
    
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        
        selection = tally[contest_name]['selections'][selection_name]
        output = "    " + selection_name
        shares = selection['shares']
        big_A = int(selection['message']['pad'])
        big_B = int(selection['message']['data'])
        
        # loop over each guardian's share of decryption
        for i in range(num_of_guardian):
            share = shares[i]
            M_i = int(share['share'])
            proof = share['proof']
            v_i = int(proof['response'])
            a_i = int(proof['pad'])
            b_i = int(proof['data'])
            c_i = int(proof['challenge'])
            k_i = k_i0_list[i]
            
            # calculate using given values
            c_i_expected = hash_elems(extended_hash, big_A, big_B, a_i, b_i, M_i)
            #print("actual c   " + str(c_i))
            #print("expected c " + str(c_i_expected))
            equ_1_left = pow(g, v_i, p)
            equ_1_right = mod_p(mod_p(a_i) * pow(k_i, c_i, p))
            equ_2_left = pow(big_A, v_i, p)
            equ_2_right = mod_p(mod_p(b_i) * pow(M_i, c_i, p))
            #print("equation1 left   " + str(equ_1_left))
            #print("equation1 right   " + str(equ_1_right))
            
            # confirm that v_i is in the set of Z_q
            if not is_valid_of_Zq(v_i):
                error = True
                output += " v_i error " + str(error)
                
            # confirm that a_i and b_i are both in the set Z_r_q
            if not is_valid_of_Zrp(a_i):
                error = True
                output += " a_i error " + str(error)
            if not is_valid_of_Zrp(b_i):
                error = True
                output += " b_i error " + str(error)
                
            # confirm challege value c_i = H(q_bar, (A,B), (a_i, b_i), M_i)
            if not equals(c_i, c_i_expected):
                error = True
                output += " c_i error " + str(error)
                
            # check first equation, g^v_i = a_i * K_i^c_i
            if not equals(equ_1_left, equ_1_right):
                error = True
                output += " equation1 error " + str(error)
                
            # check second equation, A^v_i = b_i * M_i^c_i
            if not equals(equ_2_left, equ_2_right):
                error = True
                output += " equation2 error " + str(error)
            
        if not error:
            output += " success"

        print(output)
        
    if not error: 
        print (contest_name + " success")
        
if not error:
    print("all contests all selections success")

    barchi-hallaren-selection success
    cramer-vuocolo-selection success
    court-blumhardt-selection success
    boone-lian-selection success
    hildebrand-garritty-selection success
    patterson-lariviere-selection success
    write-in-selection-president success
president-vice-president-contest success
    franz-selection success
    harris-selection success
    bargmann-selection success
    abcock-selection success
    steel-loy-selection success
    sharp-selection success
    walace-selection success
    williams-selection success
    alpern-selection success
    windbeck-selection success
    sharp-althea-selection success
    greher-selection success
    alexander-selection success
    mitchell-selection success
    lee-selection success
    ash-selection success
    kennedy-selection success
    jackson-selection success
    brown-selection success
    teller-selection success
    ward-selection success
    murphy-selection success
    newman-selection success
    callan

### green box 9 - tally verification

In [117]:
# green box 9 
error = False

for contest_name in contest_names: 
    # get all the selection names in a contest
    selection_names = list(tally[contest_name]['selections'].keys())
    #print(contest_name) 
    
    # loop over each selection within a contest
    for selection_name in selection_names:
        #print("     " + selection_name)
        m_product = 1
        selection = tally[contest_name]['selections'][selection_name]
        shares = selection['shares']
        big_B = int(selection['message']['data'])
        big_M = int(selection['value'])
        t = int(selection['tally']) 
        
        for share in shares: 
            M_i = int(share['share'])
            m_product *= mod_p(M_i)
            
        # check equation B = M * (M_i products) mod p
        if not equals(mod_p(big_B), mod_p(big_M * m_product)):
            error = True
            print(error)
            
        # check equation M = g^t mod p
        if not equals(big_M, pow(g, t, p)):
            error = True
            print(error)
if not error:
    print("[box 9] success")

[box 9] success


### green box 10 - spoiled ballot

In [ ]:
# confirm each non-dummy option in each ballot, do second part of box 6
for file in glob.glob(spoiled_ballots_folder_path + "*.json"):
    